# Импорт используемых библиотек

In [40]:
!pip install rdflib
!pip install SPARQLWrapper
!apt-get install netbase

from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS
from SPARQLWrapper import SPARQLWrapper, JSON

Reading package lists... Done
Building dependency tree       
Reading state information... Done
netbase is already the newest version (5.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


# Classes URI

In [85]:
musicalGroup = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#MusicalGroup')
rockGroup = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#RockGroup')
rapGroup = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#RapGroup')
popGroup = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#PopGroup')

musicalGenre = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#MusicalGenre')
rockMusic = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#RockMusic')
popMusic = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#PopMusic')
hipHopMusic = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#HipHopMusic')

musician = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#Musician')
award = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#Award')
country = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#Country')
language = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#Language')

# Object Properties URI

In [86]:
bornIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#bornIn')
foundedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#foundedIn')
hasDeserved = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#hasDeserved')
hasGenre = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#hasGenre')
hasMember = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#hasMember')
playedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#playedIn')
speaksIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#speaksIn')

# Data Properties URI

In [87]:
careerStarted = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#careerStarted')
careerOver = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#careerOver')
diedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#diedIn')
disbandedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#disbandedIn')
foundedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#foundedIn')
wasBorn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#wasBorn')

In [157]:
graph = Graph().parse('MusicBands.owl', format='application/rdf+xml')

# Подключение к WikiData

## добавление музыкальных групп

In [156]:
def checkIfHasLabel(s):
    if s[0] != "Q":
        return True 
    try: 
        int(s)
        return True
    except ValueError:
        return False

def add_to_graph(prop: str, id: str, uri: URIRef, ):
  sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

  sparql.setQuery("""
  SELECT ?item ?itemLabel 
  WHERE 
  { 
    ?item wdt:""" + prop + """ wd:""" + id + """.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                             ?item rdfs:label ?itemLabel }
  }
  """)
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  i = 0
  for result in results["results"]["bindings"]:
    name = result["itemLabel"]["value"]
    if not checkIfHasLabel(name):
      continue
    name = name.replace(" ", "_").replace('"','').replace('^', '').replace('`', '').replace('>', '').replace('<', '')
    item = URIRef(f'http://www.musicbands.com/ontologies/MusicBands.owl#{ name }')
    graph.add((item, RDF.type, uri))
    i = i + 1
  return i

In [158]:
add_to_graph("P31", "Q5741069", rockGroup)
add_to_graph("P31", "Q19464263", rapGroup)
add_to_graph("P31", "Q2491498", popGroup)

add_to_graph("P279", "Q11399", rockMusic)
add_to_graph("P279", "Q11401", hipHopMusic)
add_to_graph("P279", "Q37073", popMusic)

add_to_graph("P31", "Q6256", country)
add_to_graph("P106", "Q639669", musician)
add_to_graph("P31", "Q33742", language)
add_to_graph("P31", "Q1364556", award) 

358

In [ ]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql.setQuery("""
SELECT ?item 
       ?itemLabel 
       (GROUP_CONCAT(DISTINCT ?genreLabel; SEPARATOR = ", ") AS ?genres)
       (GROUP_CONCAT(DISTINCT ?work_period__start_; SEPARATOR = ", ") AS ?careerStarted) 
       (GROUP_CONCAT(DISTINCT ?work_period__end_; SEPARATOR = ", ") AS ?careerEnded)        
       (GROUP_CONCAT(DISTINCT ?languages_spoken__written_or_signed; SEPARATOR = ", ") AS ?languages) 
       (GROUP_CONCAT(DISTINCT ?country_of_origin; SEPARATOR = ", ") AS ?country) 
       (GROUP_CONCAT(DISTINCT ?award_received; SEPARATOR = ", ") AS ?awards) WHERE {
  ?item wdt:P31 wd:Q215380.
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
    ?item rdfs:label ?itemLabel.
    ?genre rdfs:label ?genreLabel.
  }
  OPTIONAL { ?item wdt:P136 ?genre. }
  OPTIONAL { ?item wdt:P2031 ?work_period__start_. }
  OPTIONAL { ?item wdt:P2032 ?work_period__end_. }
  OPTIONAL { ?item wdt:P1412 ?languages_spoken__written_or_signed. }
  OPTIONAL { ?item wdt:P495 ?country_of_origin. }
  OPTIONAL { ?item wdt:P166 ?award_received. }
}
GROUP BY ?item ?itemLabel
""")



sparql.setReturnFormat(JSON)
results = sparql.query().convert()


HTTPError: ignored

Сохранить в файл "MusicBands_populated.rdf"

In [159]:
graph.serialize(destination='MusicBands_populated.rdf')